In [ ]:
!unzip /content/results.zip

Archive:  /content/results.zip
   creating: results/
  inflating: results/iou_results_sweep_10_yolo11n_sz800_b16_lr0.01.csv  
  inflating: results/iou_results_sweep_10_yolo11n_sz800_b16_lr0.01_detailed.csv  
  inflating: results/iou_results_sweep_11_yolo11n_sz1024_b8_lr0.01.csv  
  inflating: results/iou_results_sweep_11_yolo11n_sz1024_b8_lr0.01_detailed.csv  
  inflating: results/iou_results_sweep_12_yolo11n_sz1024_b16_lr0.01.csv  
  inflating: results/iou_results_sweep_12_yolo11n_sz1024_b16_lr0.01_detailed.csv  
  inflating: results/iou_results_sweep_13_yolo11s_sz640_b8_lr0.001.csv  
  inflating: results/iou_results_sweep_13_yolo11s_sz640_b8_lr0.001_detailed.csv  
  inflating: results/iou_results_sweep_14_yolo11s_sz640_b16_lr0.001.csv  
  inflating: results/iou_results_sweep_14_yolo11s_sz640_b16_lr0.001_detailed.csv  
  inflating: results/iou_results_sweep_15_yolo11s_sz800_b8_lr0.001.csv  
  inflating: results/iou_results_sweep_15_yolo11s_sz800_b8_lr0.001_detailed.csv  
  inflating: 

In [ ]:
import pandas as pd
import os
from pathlib import Path

def calc_meanious(folder_path):


    folder = Path(folder_path)


    results = {
        'file_ious': {},
        'errors': [],
        'total_files': 0
    }


    for csv_file in folder.glob('*.csv'):
        results['total_files'] += 1

        try:
            df = pd.read_csv(csv_file)


            if 'iou' not in df.columns:
                print(f"Warning: 'iou' column not found in {csv_file.name}")
                results['errors'].append(f"{csv_file.name}: Missing 'iou' column")
                continue


            file_ious = df['iou'].dropna()
            if len(file_ious) > 0:
                mean_iou = file_ious.mean()

                config = config_info(csv_file.name)
                results['file_ious'][csv_file.name] = {
                    'mean_iou': mean_iou,
                    'num_values': len(file_ious),
                    'model': config['model'],
                    'size': config['size'],
                    'batch': config['batch'],
                    'lr': config['lr']
                }
            else:
                print(f"Warning: No valid IOU values found in {csv_file.name}")
                results['errors'].append(f"{csv_file.name}: No valid IOU values")

        except Exception as e:
            print(f"Error processing {csv_file.name}: {str(e)}")
            results['errors'].append(f"{csv_file.name}: {str(e)}")
            continue

    return results

def config_info(filename):
    parts = filename.split('_')
    config = {
        'model': 'unknown',
        'size': 'unknown',
        'batch': 'unknown',
        'lr': 'unknown'
    }

    for i, part in enumerate(parts):
        if 'yolo' in part.lower():
            config['model'] = part
        elif 'sz' in part.lower():
            config['size'] = part.replace('sz', '')
        elif 'b' in part and part[0] == 'b' and part[1:].isdigit():
            config['batch'] = part
        elif 'lr' in part.lower():
            config['lr'] = part.replace('lr', '')

    return config


if __name__ == "__main__":

    folder_path = "/content/results"

    results = calc_meanious(folder_path)


    sort_res = sorted(
        results['file_ious'].items(),
        key=lambda x: (x[1]['model'], -x[1]['mean_iou'])
    )


    print("\n results by model:")
    current_model = None
    for filename, data in sort_res:
        if data['model'] != current_model:
            current_model = data['model']
            print(f"\n{current_model.upper()}:")
            print("-" * 50)

        print(f"File: {filename}")
        print(f"mean IOU: {data['mean_iou']:.4f}")
        print(f"size: {data['size']}, Batch: {data['batch']}, Learning Rate: {data['lr']}")
        print(f"number of IOU values: {data['num_values']}")
        print("-" * 50)


    print("\n best one (5) ")
    print("=" * 70)
    top_5 = sorted(results['file_ious'].items(), key=lambda x: -x[1]['mean_iou'])[:5]
    for i, (filename, data) in enumerate(top_5, 1):
        print(f"\n{i}. mean IOU: {data['mean_iou']:.4f}")
        print(f"   model: {data['model']}")
        print(f"   config:")
        print(f"   - input size: {data['size']}")
        print(f"   - batch size: {data['batch']}")
        print(f"   - learning rate: {data['lr']}")
        print(f"   file: {filename}")
    print("\n" + "=" * 70)



    print(f"total files : {results['total_files']}")
    print(f"successfully processed files: {len(results['file_ious'])}")
    print(f"files with errors: {len(results['errors'])}")

    if results['errors']:
        print("\n errors:")
        for error in results['errors']:
            print(f"- {error}")


 results by model:

YOLO11L:
--------------------------------------------------
File: iou_results_sweep_37_yolo11l_sz640_b8_lr0.001_detailed.csv
mean IOU: 0.5300
size: 640, Batch: b8, Learning Rate: 0.001
number of IOU values: 1335
--------------------------------------------------
File: iou_results_sweep_37_yolo11l_sz640_b8_lr0.001.csv
mean IOU: 0.5300
size: 640, Batch: b8, Learning Rate: 0.001.csv
number of IOU values: 1335
--------------------------------------------------
File: iou_results_sweep_43_yolo11l_sz640_b8_lr0.01_detailed.csv
mean IOU: 0.5300
size: 640, Batch: b8, Learning Rate: 0.01
number of IOU values: 1335
--------------------------------------------------
File: iou_results_sweep_43_yolo11l_sz640_b8_lr0.01.csv
mean IOU: 0.5300
size: 640, Batch: b8, Learning Rate: 0.01.csv
number of IOU values: 1335
--------------------------------------------------
File: iou_results_sweep_44_yolo11l_sz640_b16_lr0.01_detailed.csv
mean IOU: 0.4867
size: 640, Batch: b16, Learning Rate: 0